In [ ]:
import pandas as pd
import torch
import numpy as np
import os
import sys
sys.path.append('../../')
from utils import *
import pickle

In [92]:
hiera, _label_dict, r_hiera, depths = get_hierarchy_info('./nyt.taxonomy')

In [94]:
depths

{'Top/Features': 1,
 'Top/Opinion': 1,
 'Top/Classifieds': 1,
 'Top/News': 1,
 'Top/News/Education': 2,
 'Top/News/Corrections': 2,
 'Top/News/Front Page': 2,
 'Top/News/Sports': 2,
 'Top/News/World': 2,
 'Top/News/Washington': 2,
 'Top/News/Health': 2,
 'Top/News/Obituaries': 2,
 'Top/News/Business': 2,
 'Top/News/U.S.': 2,
 'Top/News/Science': 2,
 'Top/News/Technology': 2,
 'Top/News/New York and Region': 2,
 'Top/News/World/Middle East': 3,
 'Top/News/World/Africa': 3,
 'Top/News/World/Asia Pacific': 3,
 'Top/News/World/Europe': 3,
 'Top/News/World/Countries and Territories': 3,
 'Top/News/World/Americas': 3,
 'Top/News/World/Countries and Territories/China': 4,
 'Top/News/World/Countries and Territories/Japan': 4,
 'Top/News/World/Countries and Territories/Russia': 4,
 'Top/News/World/Countries and Territories/Russia and the Former Soviet Union': 4,
 'Top/News/World/Countries and Territories/United Kingdom': 4,
 'Top/News/World/Countries and Territories/Israel': 4,
 'Top/News/World

In [93]:
_label_dict

{'Top/Features': 0,
 'Top/Opinion': 1,
 'Top/Classifieds': 2,
 'Top/News': 3,
 'Top/News/Education': 4,
 'Top/News/Corrections': 5,
 'Top/News/Front Page': 6,
 'Top/News/Sports': 7,
 'Top/News/World': 8,
 'Top/News/Washington': 9,
 'Top/News/Health': 10,
 'Top/News/Obituaries': 11,
 'Top/News/Business': 12,
 'Top/News/U.S.': 13,
 'Top/News/Science': 14,
 'Top/News/Technology': 15,
 'Top/News/New York and Region': 16,
 'Top/News/World/Middle East': 17,
 'Top/News/World/Africa': 18,
 'Top/News/World/Asia Pacific': 19,
 'Top/News/World/Europe': 20,
 'Top/News/World/Countries and Territories': 21,
 'Top/News/World/Americas': 22,
 'Top/News/World/Countries and Territories/China': 23,
 'Top/News/World/Countries and Territories/Japan': 24,
 'Top/News/World/Countries and Territories/Russia': 25,
 'Top/News/World/Countries and Territories/Russia and the Former Soviet Union': 26,
 'Top/News/World/Countries and Territories/United Kingdom': 27,
 'Top/News/World/Countries and Territories/Israel': 2

In [90]:
with open('./new_label_dict.pkl', 'rb') as f:
    label_dict = pickle.load(f)
label_dict = {v:k for k,v in label_dict.items()}
rev_label_dict = {v:k for k,v in label_dict.items()}

In [91]:
label_dict

{'Top/News': 0,
 'Top/News/Business': 1,
 'Top/News/New York and Region': 2,
 'Top/News/U.S./U.S. States, Territories and Possessions/New Jersey': 3,
 'Top/Classifieds': 4,
 'Top/Classifieds/Paid Death Notices': 5,
 'Top/News/U.S./U.S. States, Territories and Possessions/Connecticut': 6,
 'Top/Opinion': 7,
 'Top/Opinion/Opinion': 8,
 'Top/News/Corrections': 9,
 'Top/Opinion/Opinion/Editorials': 10,
 'Top/News/Sports': 11,
 'Top/News/Sports/Pro Basketball': 12,
 'Top/News/World': 13,
 'Top/News/World/Americas': 14,
 'Top/News/World/Countries and Territories': 15,
 'Top/Features': 16,
 'Top/Features/Travel': 17,
 'Top/Features/Travel/Guides': 18,
 'Top/Features/Travel/Guides/Destinations': 19,
 'Top/Features/Travel/Guides/Destinations/Central and South America': 20,
 'Top/News/U.S.': 21,
 'Top/News/Health': 22,
 'Top/News/Health/Diseases, Conditions, and Health Topics': 23,
 'Top/News/World/Europe': 24,
 'Top/Features/Travel/Guides/Destinations/Europe': 25,
 'Top/Features/Travel/Guides/D

In [ ]:
depths

def get_path(label, r_hiera):
    path = []
    # label_name = label_dict[label]
    while label != 'Root':
        path.insert(0, label)
        label = r_hiera[label]
    return path
# create label_path with label to its path
label_path = {}
for label in label_dict:
    label_path[label] = get_path(label, r_hiera)
label_path

In [ ]:
# create a dict which is depth -> label_path accrording to the depth
depth_label_path = {}
for label in label_path:
    depth = len(label_path[label])
    if depth not in depth_label_path:
        depth_label_path[depth] = {}
    depth_label_path[depth][label] = label_path[label]
depth_label_path

In [ ]:
depth_label_path[3]

In [ ]:
examples = ['Top/News', 'Top/News/Front Page', 'Top/Features',
  'Top/Features/Theater',
  'Top/Features/Theater/Reviews', 'Top/Features/Travel']
example_with_depth = [(example, depths[example]) for example in examples]
# sort the example_with_depth according to the depth
example_with_depth = sorted(example_with_depth, key=lambda x: x[1])
example_with_depth
# get the leaf node of the label
def get_leaf(labels):
    leaf = set()
    for label in labels:
        label = label[0]
        leaf = leaf - set(label_path[label])
        leaf.add(label)
    return list(leaf)
leaf = get_leaf(example_with_depth)
leaf

In [ ]:
path_reg_target = []
path_reg_embedding = []
embeddings = torch.randint(5, (len(label_path), )) # shape: (num_label)
print(embeddings)
for label in leaf:
    depth = depths[label]
    pos_path = label_path[label]
    pos_path = [label_dict[pos_path[i]] for i in range(len(pos_path))]

    neg_path = depth_label_path[depth] # (num_pos)
    neg_path = [neg_path[i] for i in neg_path if i != label] # (num_neg)
    neg_path = [[label_dict[_i] for _i in i] for i in neg_path] # (num_neg, num_label)
    
    neg_path = torch.from_numpy(np.array(neg_path))

    print(embeddings[pos_path])
    pos_embedding = embeddings[pos_path].float().sum(axis=0)
    neg_embedding = embeddings[neg_path].float().sum(axis=1)

    path_reg_embedding.extend(torch.cat([pos_embedding.unsqueeze(0), neg_embedding], dim=0))
    path_reg_target.extend([1] + [0] * neg_embedding.shape[0])

In [ ]:
# convert to tensor
path_reg_embedding = torch.stack(path_reg_embedding)
# path_reg_target = torch.tensor(path_reg_target)

path_reg_embedding.shape
np.where(np.array(path_reg_target) > 0)[0]

In [ ]:
label_dict